In [1]:
import pandas as pd
review_df=pd.read_csv("../Data/Reviews.csv")
review_df['review']= review_df['review'].astype(str)

In [2]:
#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize the SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize lists to store data
review_texts = []
positive_scores = []
negative_scores = []
neutral_scores = []
compound_scores = []
nps_indiv = []
nps_category = []  # New column for NPS categories

# Perform sentiment analysis and store scores in lists
for review in review_df['review']:
    vs = analyzer.polarity_scores(review)
    
    review_texts.append(review)
    positive_scores.append(vs['pos'])
    negative_scores.append(vs['neg'])
    neutral_scores.append(vs['neu'])
    compound_scores.append(vs['compound'])
    
    # Map compound scores to nps_indiv based on specified intervals
    if -1 <= vs['compound'] <= -9/11:
        nps_indiv.append(0)
    elif -9/11 < vs['compound'] <= -7/11:
        nps_indiv.append(1)
    elif -7/11 < vs['compound'] <= -5/11:
        nps_indiv.append(2)
    elif -5/11 < vs['compound'] <= -3/11:
        nps_indiv.append(3)
    elif -3/11 < vs['compound'] <= -1/11:
        nps_indiv.append(4)
    elif -1/11 < vs['compound'] <= 1/11:
        nps_indiv.append(5)
    elif 1/11 < vs['compound'] <= 3/11:
        nps_indiv.append(6)
    elif 3/11 < vs['compound'] <= 5/11:
        nps_indiv.append(7)
    elif 5/11 < vs['compound'] <= 7/11:
        nps_indiv.append(8)
    elif 7/11 < vs['compound'] <= 9/11:
        nps_indiv.append(9)
    else:
        nps_indiv.append(10)
    
    # Map nps_indiv scores to NPS categories
    if nps_indiv[-1] >= 9:  # Promoters
        nps_category.append('Promoter')
    elif nps_indiv[-1] >= 7:  # Passives
        nps_category.append('Passive')
    else:  # Detractors
        nps_category.append('Detractor')

# Add sentiment scores and NPS categories to the DataFrame
review_df['positive_scores'] = positive_scores
review_df['negative_scores'] = negative_scores
review_df['neutral_scores'] = neutral_scores
review_df['compound_scores'] = compound_scores
review_df['nps_indiv'] = nps_indiv
review_df['nps_category'] = nps_category

In [3]:
review_df.head()

,dataFrom,date,review,rating,extracted_devresp,devresp_time,positive_scores,negative_scores,neutral_scores,compound_scores,nps_indiv,nps_category
0,appStore,20240306 083637,Great banking app with attractive interest rat...,4,Hi Eugene Were thrilled to know that you like ...,20240308T025335Z,0.357,0.025,0.618,0.9571,10,Promoter
1,appStore,20221109 031316,A bank like no other no bank have such amazing...,5,Hi Dfdsa132 Were delighted that you love our S...,20221110T121209Z,0.146,0.052,0.802,0.9083,10,Promoter
2,appStore,20230909 083343,Notice that the drop in interest rate of 08 in...,1,Hi there Were sorry to make you feel that way ...,20230911T011906Z,0.203,0.043,0.754,0.9765,10,Promoter
3,appStore,20230627 040139,Sending money into my GXS account is a breeze ...,2,Hi Kenny thank you for alerting us to this iss...,20230628T090848Z,0.060,0.105,0.835,-0.8316,0,Detractor
4,appStore,20221002 041749,I have to say that the UIUX is one of the best...,5,Hi Mephosis we are happy that you like our app...,20221004T054950Z,0.134,0.070,0.796,0.6908,9,Promoter


In [4]:
rating_min = review_df['rating'].min()
rating_max = review_df['rating'].max()

print("Minimum rating:", rating_min)
print("Maximum rating:", rating_max)

# Checking the range
rating_range = rating_max - rating_min
print("Rating range:", rating_range)

Minimum rating: 1
Maximum rating: 5
Rating range: 4


In [5]:
# Create a new column 'nps_mapped' based on custom mapping from ratings to NPS categories
review_df['nps_mapped'] = review_df['rating'].map({1: 'Detractor', 2: 'Detractor', 3: 'Detractor', 4: 'Passive', 5: 'Promoter'})
# Calculate accuracy by comparing 'nps_category' and 'nps_mapped'
correct_predictions = (review_df['nps_category'] == review_df['nps_mapped']).sum()
total_predictions = len(review_df)
accuracy = (correct_predictions / total_predictions) * 100

print("Accuracy of VADER NPS Sentiment Analysis:", accuracy, "%")


Accuracy of VADER NPS Sentiment Analysis: 64.11483253588517 %
